In [216]:
import os

import cv2
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

from vedaseg.utils import Config
from vedaseg.datasets import build_dataset
from vedaseg.transforms import build_transform
from vedaseg.dataloaders import build_dataloader
from vedaseg.datasets.xray import XrayDataset
from vedaseg.dataloaders.samplers import build_sampler

# from vedaseg.runners.

In [217]:
cfg_path = 'configs/xray_seg_ks0.py'
json_path = '/DATA/home/tianhewang/DataSets/KS_X-ray/ks_0/ks_0_train.json'
cfg = Config.fromfile(cfg_path)

test_cfg = cfg['test']
train_cfg = cfg['train']
inference_cfg = cfg['inference']
common_cfg = cfg['common']

size_h = 1153
size_w = 1153
ignore_label = 255
image_pad_value = (0, 0, 0)
# print(train_cfg)
img_norm_cfg = dict(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225),
                    max_pixel_value=255.0)


transform_cfg = [
                dict(type='LongestMaxSize', h_max=size_h, w_max=size_w,
                     interpolation=cv2.INTER_LINEAR),
                dict(type='PadIfNeeded', min_height=size_h, min_width=size_w,
                     value=image_pad_value, mask_value=0),
#                 dict(type='FactorScale', scale=1/16,
#                      interpolation=cv2.INTER_LINEAR,
#                      always_apply=True,
#                      p=1.0),
                # dict(type='GaussianBlur', blur_limit=7, p=0.5),
                dict(type='Normalize', **img_norm_cfg),
                dict(type='ToTensor'),
            ]

transform = build_transform(transform_cfg)
dataset = build_dataset(train_cfg['data']['train']['dataset'],
                        dict(transform=transform))
# shuffle = train_cfg['data']['train']['dataloader'].pop('shuffle', False)

# sampler = build_sampler(False, 
#                         train_cfg['data']['train']['sampler'],
#                         dict(dataset=dataset, shuffle=shuffle))

# train_cfg['data']['train']['dataloader']['samples_per_gpu'] = 1
# train_cfg['data']['train']['dataloader']['workers_per_gpu'] = 1
# dataloader = build_dataloader(False, 
#                               1, 
#                               train_cfg['data']['train']['dataloader'],
#                               dict(dataset=dataset, sampler=sampler))

In [126]:
out_path = '/DATA/home/tianhewang/DataSets/KS_X-ray/ks_0/vis'
colors = [(0,255,0), (0,0,255)]

count = [0, 0]
for idx, (image, mask) in enumerate(
    tqdm(dataset,
         dynamic_ncols=True,
         unit='images',
         unit_scale=True)):
    image = image.numpy().astype(int)
    image = np.moveaxis(image, 0, -1)

    mask = mask.numpy().astype(int)
    for clas_ind in range(mask.shape[0]):
        current = mask[clas_ind, ...]
        current = current.reshape(-1)
        if sum(current==1) > 0:
            count[clas_ind] += 1
print(count, len(dataset))
#         image[mask[clas_ind, ...]==1]=colors[clas_ind]

    
#     target_dir = os.path.join(out_path, f'{idx}.jpg')
#     cv2.imwrite(target_dir, image)
    
#     plt.figure(figsize=(10, 8))
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()
    
#     if idx >= 5:
#         break
#     fn = data['img_metas'].data['ori_filename']
#     fn = os.path.split(fn)[-1]
#     target_dir = os.path.join(visualize_dir, fn)
#     vis = process_image(data, exclude=None,
#                         with_bbox=True)

#     vis = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(target_dir, vis)

[2278, 1580]


In [218]:
image_, mask_ = dataset[15]
# print(mask_)
for _, mask in dataset:
    print(mask.shape)
    res = torch.zeros((mask.shape[0], 1, 1))
    for clas_ind in range(mask.shape[0]):
        current = mask[clas_ind, ...].reshape(-1)
        if 1 in current:
            res[clas_ind, 0, 0] = 1
    print(res.shape)
    break


torch.Size([2, 1153, 1153])
torch.Size([2, 1, 1])


In [169]:
out_path = '/DATA/home/tianhewang/DataSets/KS_X-ray/ks_0/vis'
colors = [(0,255,0), (0,0,255)]
# m = nn.AdaptiveMaxPool2d((1, 1))

count = [0, 0]
for idx, (image, mask) in enumerate(
    tqdm(dataset,
         dynamic_ncols=True,
         unit='images',
         unit_scale=True)):
#     image = image.numpy().astype(int)
#     image = np.moveaxis(image, 0, -1)

    mask = mask.numpy().astype(int)
    for clas_ind in range(mask.shape[0]):
        current = mask[clas_ind, ...].reshape(-1)
        if 1 in current:
            count[clas_ind] += 1
print(count, len(dataset))

# full: [2278, 1582] 3279
# 16(673, 1153): [2278, 1580] 3279
# 16(705, 1153): [2278, 1578] 3279
# 16(737, 1153): [2278, 1580] 3279
# 16(1153, 1153): [2278, 1580] 3279

100%|██████████| 3.28k/3.28k [05:42<00:00, 9.58images/s]

[2278, 1580] 3279
